In [22]:
import cftime
import datetime
import eofs
import numpy as np
import xarray as xr

In [3]:
path = "/work/Katherine.Turner/tempest-extremes/odiv-251/comp_dic_stf_gas/"

In [5]:
ds = xr.open_dataset(f"{path}comp_dic_stf_gas_anom.nc", use_cftime=True)

In [32]:
start_date = cftime.DatetimeNoLeap(151, 1, 1, 12, 0, 0, 0, has_year_zero=True)

ndays = 95*365

dates = np.array([start_date + datetime.timedelta(days=idx) for idx in range(ndays)])

AttributeError: 'numpy.ndarray' object has no attribute 'dt'

In [45]:
mean_storms = np.ones(shape = (len(dates), 40, 40))

for i, d in enumerate(dates):
    idx = ds.snap_time == d
    mean_storms[i,:,:] = ds.snap_dic_stf_gas_anom[idx,:,:].mean("snapshot").data

KeyboardInterrupt: 

In [48]:
mean_storms_2 = mean_storms[:i,:,:]
dates_2 = dates[:i]

In [53]:
ds.snap_dic_stf_gas_anom

<xarray.DataArray 'snap_dic_stf_gas_anom' (snapshot: 867004, y: 40, x: 40)>
[1387206400 values with dtype=float32]
Coordinates:
  * x        (x) float64 -9.75 -9.25 -8.75 -8.25 -7.75 ... 8.25 8.75 9.25 9.75
  * y        (y) float64 -9.75 -9.25 -8.75 -8.25 -7.75 ... 8.25 8.75 9.25 9.75
Dimensions without coordinates: snapshot

In [55]:
ds_mean = xr.Dataset(
    data_vars=dict(
        flux=(["time", "y", "x"], mean_storms_2),
        ),
    coords=dict(
        x=ds.x.data,
        y=ds.y.data,
        time=dates_2,
        ),
    attrs=dict(description=f"Mean daily flux anomaly"),
    )


In [71]:
X = np.meshgrid(ds.x.data,ds.y.data)
mask = xr.DataArray(np.sqrt(X[0]**2 + X[1]**2) <= 10, dims=['x', 'y'])
masked_flux = xr.where(mask, ds_mean.flux, 0)
masked_flux = xr.where(ds_mean.flux == np.nan, 0, masked_flux)

In [72]:
test = eofs.xarray.Eof(masked_flux.transpose("time", "y", "x"), center=False)

ValueError: missing values detected in different locations at different times